In [3]:
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
from decimal import Decimal
import hashlib

# RPC Configuration
rpc_user = "nidarsana02"
rpc_password = "hjsHjZR1OIAft7Y-lYScC9F94PR54eAvp0vGPurIY1w"
rpc_host = "127.0.0.1"
rpc_port = 18443
wallet_name = "CS216"

In [4]:
# Connect to Bitcoin RPC
rpc_client = AuthServiceProxy(f"http://{rpc_user}:{rpc_password}@{rpc_host}:{rpc_port}/wallet/{wallet_name}")

# Ensure wallet is loaded
def ensure_wallet_loaded(wallet_name):
    try:
        loaded_wallets = rpc_client.listwallets()
        if wallet_name in loaded_wallets:
            print(f"Wallet '{wallet_name}' is already loaded.")
            return
        print(f"Loading wallet '{wallet_name}'...")
        rpc_client.loadwallet(wallet_name)
    except JSONRPCException as e:
        if e.error.get("code") == -18:  # Wallet does not exist
            print(f"Wallet '{wallet_name}' not found. Creating...")
            rpc_client.createwallet(wallet_name)
        else:
            raise

ensure_wallet_loaded(wallet_name)

# Generate P2SH-SegWit Addresses
address_A = rpc_client.getnewaddress("A'", "p2sh-segwit")
address_B = rpc_client.getnewaddress("B'", "p2sh-segwit")
address_C = rpc_client.getnewaddress("C'", "p2sh-segwit")

print(f"Address A: {address_A}")
print(f"Address B: {address_B}")
print(f"Address C: {address_C}")

Loading wallet 'CS216'...
Address A: 2MwyA72EFMaCzMi5dxqZ9iXe17jJ2aqpeE3
Address B: 2MvBEkaAn63PxQvdGEZ6gjBACGbyiMad1hJ
Address C: 2N2ZM6xhy4aWdxxiwsdXboLvP656WhPab4r


In [5]:
# Fund Address A
rpc_client.sendtoaddress(address_A, 0.01)
rpc_client.generatetoaddress(1, address_A)  # Confirm transaction by mining 1 block

# Get UTXO for Address A
unspent_outputs = rpc_client.listunspent(0, 9999999, [address_A])
if not unspent_outputs:
    raise ValueError("No UTXOs found for Address A.")

utxo = unspent_outputs[0]
amount = Decimal(str(utxo["amount"]))

# Create Transaction A -> B
raw_tx = rpc_client.createrawtransaction(
    [{"txid": utxo["txid"], "vout": utxo["vout"]}],
    {address_B: amount - Decimal("0.0001")}
)

In [6]:
signed_tx = rpc_client.signrawtransactionwithwallet(raw_tx)
decoded_tx = rpc_client.decoderawtransaction(signed_tx["hex"])
print("Decoded Transaction A -> B:", decoded_tx)
txid = rpc_client.sendrawtransaction(signed_tx["hex"])
print("Transaction ID A -> B:", txid)

# Get UTXO for Address B
unspent_outputs_B = rpc_client.listunspent(0, 9999999, [address_B])
if not unspent_outputs_B:
    raise ValueError("No UTXOs found for Address B.")

utxo_B = unspent_outputs_B[0]
amount_B = Decimal(str(utxo_B["amount"]))

Decoded Transaction A -> B: {'txid': '41c56c295ecb22b44d54a4814b4ab27ba59cd2bd914e81e75a45661cb1ebb2b4', 'hash': '329199a72a9e57e0128aa6e6bac4d2e39c6aee8f677e4f2080342c86f17974f0', 'version': 2, 'size': 215, 'vsize': 134, 'weight': 533, 'locktime': 0, 'vin': [{'txid': '94aa25274a78627373065aff8d6ff658b86e084b42cb4fccfe9f52d162c4311d', 'vout': 1, 'scriptSig': {'asm': '0014f026da7aa717456cbca8b5edd0028b9cfb588228', 'hex': '160014f026da7aa717456cbca8b5edd0028b9cfb588228'}, 'txinwitness': ['3044022031236247f6d608c845733539fb696f19b95e8532f30872e81c14ae0ed46594b702201f24471a940d545d5d90910d782b08e65754d9e6c8e29885d9ef32748592f4b801', '0272222cc2603ddf3b562f3b09c37e5a88538f117cf2f23bc36599fe110730d2ea'], 'sequence': 4294967293}], 'vout': [{'value': Decimal('0.00990000'), 'n': 0, 'scriptPubKey': {'asm': 'OP_HASH160 202759d62f1e37e5519f9c6f39996272b285785d OP_EQUAL', 'desc': 'addr(2MvBEkaAn63PxQvdGEZ6gjBACGbyiMad1hJ)#d9qgjund', 'hex': 'a914202759d62f1e37e5519f9c6f39996272b285785d87', 'address'

In [7]:
# Create Transaction B -> C
raw_tx_B = rpc_client.createrawtransaction(
    [{"txid": utxo_B["txid"], "vout": utxo_B["vout"]}],
    {address_C: amount_B - Decimal("0.0001")}
)

signed_tx_B = rpc_client.signrawtransactionwithwallet(raw_tx_B)
decoded_tx_B = rpc_client.decoderawtransaction(signed_tx_B["hex"])
print("Decoded Transaction B -> C:", decoded_tx_B)
txid_B = rpc_client.sendrawtransaction(signed_tx_B["hex"])
print("Transaction ID B -> C:", txid_B)

Decoded Transaction B -> C: {'txid': 'e134d6afa9978ea8441eb4ef61da28ec57ea8fce74578aa851515f909221cae0', 'hash': '6ddc5285aaaa12e4d3875cb6b9887fc413572c1ec4c1a1b2a922fc6de5e655a1', 'version': 2, 'size': 215, 'vsize': 134, 'weight': 533, 'locktime': 0, 'vin': [{'txid': '41c56c295ecb22b44d54a4814b4ab27ba59cd2bd914e81e75a45661cb1ebb2b4', 'vout': 0, 'scriptSig': {'asm': '0014e1e20a916ad6f335e671141445b65e4823b65d80', 'hex': '160014e1e20a916ad6f335e671141445b65e4823b65d80'}, 'txinwitness': ['304402204f04d7c889881f8c28bc23c434472378668d38b08b77a3b788cedeba2f8952dc02204f9ee472fd6576bde2b8007967e2df02acbb9a4fa25135c3499a50ca4270b6df01', '03f49337aab57a5489962fd97d7436bd7b60b6c73a8065f364e36cb3f1f02fb878'], 'sequence': 4294967293}], 'vout': [{'value': Decimal('0.00980000'), 'n': 0, 'scriptPubKey': {'asm': 'OP_HASH160 66268f0bd74e527ffbc2834dbfb973020d7cfaa7 OP_EQUAL', 'desc': 'addr(2N2ZM6xhy4aWdxxiwsdXboLvP656WhPab4r)#lsd0q8hz', 'hex': 'a91466268f0bd74e527ffbc2834dbfb973020d7cfaa787', 'address'

In [8]:
# Extract Public Key from Transaction B (txinwitness[1])
public_key = decoded_tx_B["vin"][0]["txinwitness"][1]  # Second witness (Public Key)

# Compute HASH160(pubKey)
pubkey_hash = hashlib.new("ripemd160", hashlib.sha256(bytes.fromhex(public_key)).digest()).hexdigest()

# Create redeem script for P2SH-P2WPKH (0014 + pubkey_hash)
redeem_script = "0014" + pubkey_hash  # Ensure correct formatting

# Compute HASH160(redeem_script)
redeem_script_hash = hashlib.new("ripemd160", hashlib.sha256(bytes.fromhex(redeem_script)).digest()).hexdigest()

# Extract Expected Redeem Script Hash from scriptPubKey
expected_redeem_script_hash = decoded_tx["vout"][0]["scriptPubKey"]["asm"].split(" ")[1]  # Extract the hash from OP_HASH160

# Check if the computed hash matches the expected hash
if redeem_script_hash == expected_redeem_script_hash:
    print("✅ Hash matches! Transaction is valid.")
else:
    print("❌ Hash mismatch! Check public key and script format.")

# Debugging Info
print(f"Public Key: {public_key}")
print(f"Computed Redeem Script Hash: {redeem_script_hash}")
print(f"Expected Redeem Script Hash: {expected_redeem_script_hash}")


✅ Hash matches! Transaction is valid.
Public Key: 03f49337aab57a5489962fd97d7436bd7b60b6c73a8065f364e36cb3f1f02fb878
Computed Redeem Script Hash: 202759d62f1e37e5519f9c6f39996272b285785d
Expected Redeem Script Hash: 202759d62f1e37e5519f9c6f39996272b285785d
